<h1> Outline </h1>

- array of acceptable genres
- Input: 10 games --> all games w/in these genres, w/in accepted
- --> console as filter --> genres as filter, queried data - keywords mlo
- query: {name:keywords} (kelly will do this using postman)
- data in: name, keywords
- process: calculate dist as # common keywords, to keywords in games user chose
- output: closest games to target keywords

<h1> Code </h1>

In [33]:
%matplotlib inline
import numpy as np
import pandas as pd
import random
#pd.options.display.max_rows = 25
#pd.options.display.max_columns = 30


#games = pd.read_csv("IGDB_games.csv", sep=",")

In [34]:
ids_with_genres = pd.read_csv("ids_with_genres.csv", sep=",")
ids_with_genres

,id,name
0,4,Fighting
1,5,Shooter
2,7,Music
3,8,Platform
4,9,Puzzle
5,10,Racing
6,11,Real Time Strategy (RTS)
7,12,Role-playing (RPG)
8,13,Simulator
9,14,Sport


<h1> example with fake data </h1>

In [35]:
d = {'name': ['Skyrim', 'Destiny', 'Mario Kart', 'Forza'], 'adventure': [1, 0,0,0], 'racing': [0, 0,1,1], 'shooter': [0, 1,0,0]}
df = pd.DataFrame(data=d)
df

,name,adventure,racing,shooter
0,Skyrim,1,0,0
1,Destiny,0,0,1
2,Mario Kart,0,1,0
3,Forza,0,1,0


In [36]:
features = ["adventure", "racing",
            "shooter"]

X_train_dict = df[features].to_dict(orient="records")
y_train = df["name"]

In [37]:
X_train = df[["adventure", "racing", "shooter"]]
y_train = df["name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [38]:
x_new = pd.Series(dtype=int)
x_new["adventure"] = 0
x_new["racing"] = 1
x_new["shooter"] = 0

x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

adventure   -0.500000
racing       0.866025
shooter     -0.500000
dtype: float64

In [39]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()
dists_sorted

2    0.000000
3    0.000000
0    2.645751
1    2.645751
dtype: float64

In [40]:
inds_nearest = dists_sorted.index[:9]
inds_nearest

Int64Index([2, 3, 0, 1], dtype='int64')

In [41]:
df.loc[inds_nearest]

,name,adventure,racing,shooter
2,Mario Kart,0,1,0
3,Forza,0,1,0
0,Skyrim,1,0,0
1,Destiny,0,0,1


<h1> Example with 100 gamesread from CSV </h1>

In [42]:
# read the csv with all the games
games = pd.read_csv("test_dataset.csv", sep=",")

# split the string into a list of strings
games.genres = games.genres.str.split(', ').to_frame()

# fill in [0] for all the NaN values
games['genres'] = games['genres'].fillna({i: [0] for i in df.index})
games

,id,category,created_at,external_games,keywords,name,similar_games,slug,tags,updated_at,...,hypes,dlcs,bundles,expansions,expanded_games,version_parent,version_title,remakes,standalone_expansions,remasters
0,73817,0,10/5/17 0:00,"[147971, 1870903]","[1097, 1219, 2396, 4260, 5570]",Spy Snatcher,"[18290, 58702, 58867, 59500, 59720, 79064, 800...",spy-snatcher,"[536872009, 536872131, 536873308, 536875172, 5...",8/26/20 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,112119,0,11/16/18 0:00,"[1616043, 1911731]",NaN,Woohoojin Bridges,NaN,woohoojin-bridges,NaN,8/26/20 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,134130,0,5/24/20 0:00,[1976473],NaN,Astr√•,NaN,astra,NaN,11/6/20 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21667,0,7/28/16 22:40,"[115710, 220247, 1923649, 2006171]","[350, 934, 4134, 4144, 4428, 4841, 4964, 5315,...",Siesta Fiesta,"[20329, 25646, 28070, 36198, 55173, 55190, 559...",siesta-fiesta,"[1, 27, 268435465, 268435486, 268435487, 53687...",10/5/21 22:26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,172773,0,10/1/21 17:42,[2127350],[19521],Legend Of Yashiyu 2,"[54775, 69007, 96217, 99118, 103303, 106987, 1...",legend-of-yashiyu-2,"[17, 27, 268435468, 536890433]",10/1/21 18:16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,153116,1,6/21/21 18:15,[2013517],NaN,Trainz Railroad Simulator 2019: GT GP9 2 Pack,"[28277, 65827, 87485, 100794, 100806, 101573, ...",trainz-railroad-simulator-2019-gt-gp9-2-pack,[268435469],7/26/21 9:44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,112972,0,12/4/18 11:03,"[1684187, 1993025]",NaN,Plat4mer,"[32902, 37419, 40524, 76263, 96217, 105233, 11...",plat4mer,"[1, 268435488]",7/26/21 3:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,62297,0,9/10/17 0:00,"[115891, 1900701]","[26, 27]",Legends Football '98,"[701, 15851, 18681, 57738, 100359, 103377, 106...",legends-football-98,"[268435470, 536870938, 536870939]",8/26/20 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,86456,0,2/6/18 0:00,[1977663],NaN,Battlezone 2 Remastered,NaN,battlezone-2-remastered,NaN,11/6/20 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# convert the list of strings into a list of ints
for i, genre_list in enumerate(games.genres):
    for genre in genre_list:
        games.genres[i] = list(map(int, games.genres[i]))

<ipython-input-43-01529eb85361>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games.genres[i] = list(map(int, games.genres[i]))


In [44]:
# Getting distinct genre types for generating columns of genre type.
genre_columns = list(set([j for i in games['genres'].tolist() for j in i]))

In [45]:
# copy over name and id columns to new df
selected_columns = games[["id","name"]]
genre_df = selected_columns.copy()

In [46]:
# Iterating over every list to create and fill values into columns.
for j in genre_columns:
    genre_df[j] = 0
for i in range(games.shape[0]):
    for j in genre_columns:
        if(j in games['genres'].iloc[i]):
            genre_df.loc[i,j] = 1

In [47]:
# Features in our model. All quantitative, except Neighborhood.
features = genre_columns

X_train_dict = genre_df[features].to_dict(orient="records")
y_train = genre_df["name"]

In [48]:
X_train = genre_df[genre_columns]
y_train = genre_df["name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [49]:
x_new = pd.Series(0, index=genre_columns)

x_new[0] = 0
x_new[9] = 1
x_new[30] = 1
x_new[31] = 1

x_new_sc = (x_new - X_train.mean()) / X_train.std()

In [50]:
x_new = pd.Series(0, index=genre_columns)

x_new[9] = 1
x_new[30] = 1
x_new[31] = 1

x_new_sc = (x_new - X_train.mean()) / X_train.std()

In [51]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()

In [52]:
inds_nearest = dists_sorted.index[:9]

In [53]:
genre_df.loc[inds_nearest].head()

,id,name,0,2,4,5,8,9,10,12,...,15,16,24,25,30,31,32,33,34,35
3,21667,Siesta Fiesta,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,0
32,176456,Captain Hookfoot,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
23,141409,Mirage,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
76,176483,Droplet,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
60,79272,Ballochet 3,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1>Example reading from json result of query</h1>

In [54]:
df = pd.read_json('input_data_example.json')
df

,id,genres,name,themes
0,121,"[{'id': 31, 'name': 'Adventure'}, {'id': 33, '...",Minecraft,"[{'id': 17, 'name': 'Fantasy'}, {'id': 21, 'na..."
1,1879,"[{'id': 8, 'name': 'Platform'}, {'id': 12, 'na...",Terraria,"[{'id': 1, 'name': 'Action'}, {'id': 18, 'name..."
2,8173,"[{'id': 5, 'name': 'Shooter'}, {'id': 15, 'nam...",Overwatch,"[{'id': 1, 'name': 'Action'}, {'id': 18, 'name..."
3,9066,"[{'id': 13, 'name': 'Simulator'}, {'id': 15, '...",Cities: Skylines,"[{'id': 28, 'name': 'Business'}, {'id': 33, 'n..."
4,9527,"[{'id': 8, 'name': 'Platform'}, {'id': 10, 'na...",SpeedRunners,"[{'id': 1, 'name': 'Action'}, {'id': 40, 'name..."
5,135400,"[{'id': 13, 'name': 'Simulator'}, {'id': 31, '...",Minecraft,"[{'id': 17, 'name': 'Fantasy'}, {'id': 21, 'na..."


In [55]:
genre_list = []
theme_list = []
Dict = {}
# for each game in the database
for i, game in df.iterrows():
    
    # add each genre of that game to a list
    for j, genre in enumerate(game.genres):
        add_genre = game.genres[j]['id']
        genre_list.append(add_genre)

    # add each theme of that game to a list
    for jj, theme in enumerate(game.themes):
        add_theme = game.themes[jj]['id']
        theme_list.append(add_theme)
    
    # create a dictionary of names, ids, and the genre and theme lists    
    Dict[i] = {'game_name': df['name'][i], 'game_id':df['id'][i],'genre_list':genre_list, 'theme_list':theme_list}
    # empty the to be added lists for the next game
    genre_list = []
    theme_list = []

# convert dict to df    
ggt_df = pd.DataFrame.from_dict(Dict,orient='index')
ggt_df

,game_name,game_id,genre_list,theme_list
0,Minecraft,121,"[31, 33]","[17, 21, 33, 38, 41]"
1,Terraria,1879,"[8, 12, 13, 15, 31, 32]","[1, 18, 19, 21, 33, 38]"
2,Overwatch,8173,"[5, 15]","[1, 18, 27]"
3,Cities: Skylines,9066,"[13, 15]","[28, 33]"
4,SpeedRunners,9527,"[8, 10, 14, 32, 33]","[1, 40]"
5,Minecraft,135400,"[13, 31, 33]","[17, 21, 33, 38, 41]"


In [56]:
# get list of all genres
genre_columns = list(set([j for i in ggt_df['genre_list'].tolist() for j in i]))

# copy over name and id columns to new df
selected_columns = ggt_df[["game_name","game_id"]]
choice_genre = selected_columns.copy()

In [57]:
# Iterating over every list to create and fill values into columns for the game database
for j in genre_columns:
    choice_genre[j] = 0
for i in range(ggt_df.shape[0]):
    for j in genre_columns:
        if(j in ggt_df['genre_list'].iloc[i]):
            choice_genre.loc[i,j] = 1
            choice_genre.fillna(0, inplace=True)
choice_genre

,game_name,game_id,32,33,5,8,10,12,13,14,15,31
0,Minecraft,121,0,1,0,0,0,0,0,0,0,1
1,Terraria,1879,1,0,0,1,0,1,1,0,1,1
2,Overwatch,8173,0,0,1,0,0,0,0,0,1,0
3,Cities: Skylines,9066,0,0,0,0,0,0,1,0,1,0
4,SpeedRunners,9527,1,1,0,1,1,0,0,1,0,0
5,Minecraft,135400,0,1,0,0,0,0,1,0,0,1


In [58]:
# extracting info
features = genre_columns

X_train_dict = choice_genre[features].to_dict(orient="records")
y_train = choice_genre["game_name"]

In [59]:
# extracting info
X_train = choice_genre[genre_columns]
y_train = choice_genre["game_name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [60]:
# simulate user selecting a game
choice = ggt_df.loc[ggt_df['game_id'] == choice_genre.iloc[random.randint(0,5)].game_id]
print("User chose game " + choice['game_name'].to_string())


# create a new df where we will put info from the user's input
input_df = pd.DataFrame(columns=genre_columns, index=[0])


# Iterating over every list to create and fill values into columns for user input
for j in genre_columns:
    input_df[j] = 0
for i in range(choice.shape[0]):
    for j in genre_columns:
        if(j in choice['genre_list'].iloc[i]):
            input_df.loc[i,j] = 1

# store this in x_new
x_new = input_df.transpose().squeeze()

User chose game 3    Cities: Skylines


In [61]:
# Features in our model
features = genre_columns

X_train_dict = choice_genre[features].to_dict(orient="records")
y_train = choice_genre["game_name"]

In [62]:
X_train = choice_genre[genre_columns]
y_train = choice_genre["game_name"]

X_train_sc = (X_train - X_train.mean()) / X_train.std()

In [63]:
x_new_sc = (x_new - X_train.mean()) / X_train.std()
x_new_sc

32   -0.645497
33   -0.912871
5    -0.408248
8    -0.645497
10   -0.408248
12   -0.408248
13    0.912871
14   -0.408248
15    0.912871
31   -0.912871
dtype: float64

In [64]:
dists = np.sqrt(((X_train_sc - x_new_sc) ** 2).sum(axis=1))
dists_sorted = dists.sort_values()

inds_nearest = dists_sorted.index[:9]

choice_genre.loc[inds_nearest]

,game_name,game_id,32,33,5,8,10,12,13,14,15,31
3,Cities: Skylines,9066,0,0,0,0,0,0,1,0,1,0
2,Overwatch,8173,0,0,1,0,0,0,0,0,1,0
5,Minecraft,135400,0,1,0,0,0,0,1,0,0,1
0,Minecraft,121,0,1,0,0,0,0,0,0,0,1
1,Terraria,1879,1,0,0,1,0,1,1,0,1,1
4,SpeedRunners,9527,1,1,0,1,1,0,0,1,0,0
